In [57]:
from datasets import load_dataset

ds = load_dataset("K-Monitor/kmdb_base")

Generating train split: 100%|██████████| 59180/59180 [00:01<00:00, 55769.09 examples/s]


In [61]:
from collections import defaultdict
from datetime import datetime
import pandas as pd

def process_articles(article_list):
    # Negyedéves adatok tárolása
    quarterly_data = defaultdict(lambda: defaultdict(int))

    for article in article_list:
        # Publikáció időpontjának ellenőrzése és beolvasása
        pub_time = article.get('pub_time', '')
        if not pub_time:
            continue
        
        try:
            pub_datetime = datetime.strptime(pub_time, '%Y-%m-%d %H:%M:%S')
        except ValueError:
            print(f"Warning: Date format mismatch for entry with news_id {article['news_id']}")
            continue

        # Év és negyedév számítása
        year = pub_datetime.year
        quarter = (pub_datetime.month - 1) // 3 + 1

        # Az aktához tartozó fájl lekérdezése
        archive_name = article['files'][0] if article.get('files') else 'Unknown'

        # Cikkek számának növelése 
        quarterly_data[archive_name][(year, quarter)] += 1

    # Az adatok átalakítása Pandas DataFrame-be
    df = pd.DataFrame(quarterly_data).T.fillna(0).astype(int)

    # Oszlopok elnevezése és rendezése időrend szerint
    df.columns = [f'{year} Q{quarter}' for year, quarter in df.columns]
    df = df.reindex(sorted(df.columns, key=lambda x: (int(x.split()[0]), int(x.split()[1][1]))), axis=1)

    return df


df = process_articles(ds['train'])

print(df)

                                                    1970 Q1  1996 Q4  1997 Q1  \
Kopaszi-gát                                               0        0        0   
Unknown                                                   7        2        0   
Kulcsár-ügy                                               0        0        0   
Nyírfa-dosszié                                            0        0        0   
Malom Center                                              0        0        0   
...                                                     ...      ...      ...   
Lombkorona sétány                                         0        0        0   
Soroksári ipari beruházás                                 0        0        0   
Maxi-Dubaj Rákosrendezőn                                  0        0        0   
Fiktív szerződések a III. kerületi önkormányzatnál        0        0        0   
Pázmány Campus a belvárosban                              0        0        0   

                           

In [62]:
df.to_csv('akta_bontas.csv')